In [1]:
# ! pip install optbinning

In [2]:
import joblib
import numpy as np
import pandas as pd
import gc
import time
import os
import sys
from contextlib import contextmanager
from lightgbm import LGBMClassifier
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
# from bayes_opt import BayesianOptimization
# from skopt import BayesSearchCV 
from optbinning import OptimalBinning
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
# from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA
from functools import reduce
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import random
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
pd.set_option('display.max_rows', 500)

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(1)

In [5]:
# read data
train = pd.read_csv('c:/Users/user/Desktop/oil/data/input/train.csv')
test = pd.read_csv('c:/Users/user/Desktop/oil/data/input/test.csv')
sample_submission = pd.read_csv('c:/Users/user/Desktop/oil/data/input/sample_submission.csv')
original_columns = test.columns.tolist()

In [6]:
# test데이터셋에 있는 변수만 선택 
train = train[test.columns.to_list()+['Y_LABEL','SAMPLE_TRANSFER_DAY']].copy()

> 파생변수 생성

In [7]:
"""
new features 
"""

train['YEAR_COMPONENT'] = train[['YEAR','COMPONENT_ARBITRARY']].apply(lambda x: '-'.join(x.astype(str)),axis=1)
test['YEAR_COMPONENT'] = test[['YEAR','COMPONENT_ARBITRARY']].apply(lambda x: '-'.join(x.astype(str)),axis=1)

var = [i for i in test.columns.tolist() if i not in ['ID','COMPONENT_ARBITRARY']]
train['zero_cnt'] = train[var].apply(lambda x:(x==0).sum(),axis=1)
test['zero_cnt'] = test[var].apply(lambda x:(x==0).sum(),axis=1)

In [8]:
"""
transformation 
"""

for var in ['ANONYMOUS_1','PQINDEX','V40','ZN','FE']:
    
    train[var+'_log1'] = np.log(train[var]+1)
    test[var+'_log1'] = np.log(test[var]+1)

In [9]:
"""
normalization
""" 

for var in ['ZN','FE','CU','V40','MO','CR','NI','ANONYMOUS_1','ANONYMOUS_2']:
    
    ZN_min = train[var].min()
    ZN_max = train[var].max()
    train[var] = (train[var] - ZN_min) / (ZN_max-ZN_min)
    test[var] = (test[var] - ZN_min) / (ZN_max-ZN_min)

In [10]:
"""
feature combination
"""

seed_everything(1)

from itertools import chain, product 
candidate_var = ['ZN','FE','CU','V40','MO','CR','NI','ANONYMOUS_1','ANONYMOUS_2']
pairs = list(chain(product(candidate_var, candidate_var), product(candidate_var, candidate_var))) 
pairs = pd.Series([sorted([i,j]) for (i,j) in set(pairs) if i!=j]).drop_duplicates().reset_index(drop=True).tolist()
pairs = sorted(pairs)
# pairs = pairs.sort_values()
# random.shuffle(pairs)
# print(pairs)

for i in range(len(pairs)):

    train[pairs[i][0]+'_'+pairs[i][1]] = train[pairs[i][0]] * train[pairs[i][1]]
    test[pairs[i][0]+'_'+pairs[i][1]] = test[pairs[i][0]] * test[pairs[i][1]] 

In [11]:
"""
woe features 
"""

for variable in ['ANONYMOUS_1','PQINDEX','V40','ZN','FE']+['ANONYMOUS_1_V40', 'ANONYMOUS_1_CR', 'ANONYMOUS_1_ZN','ANONYMOUS_1_FE']:

    optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

    x = train[variable].values
    y = train['Y_LABEL']

    optb.fit(x, y)

    binning_table = optb.binning_table

    a1 = binning_table.build()
    
    # check 
    # print('iv:',a1['IV'].sum())
    # display(a1)
    # binning_table.plot(metric="event_rate")

    train[variable+'_woe_bin'] = pd.cut(train[variable],bins=optb.splits.tolist()).astype(str)
    test[variable+'_woe_bin'] = pd.cut(test[variable],bins=optb.splits.tolist()).astype(str)

In [12]:
"""
group operations
"""

opt_numeric_var = ['ANONYMOUS_1_V40', 'ANONYMOUS_1_CR', 'ANONYMOUS_1_ZN','ANONYMOUS_1_FE']

for group in ['COMPONENT_ARBITRARY','YEAR_COMPONENT','YEAR']+['FE_woe_bin']:

    for numeric_var in ['ZN','FE','V40','PQINDEX','CU','ANONYMOUS_1','ANONYMOUS_2']+opt_numeric_var:
        
        train = train.copy()
        test = test.copy()

        a1 = train.groupby([group])[numeric_var].median().to_dict()

        train[numeric_var+'_'+group+'_'+'median'] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+'median'] = test[group].map(a1)

        train[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'median'] = train[numeric_var] - train[numeric_var+'_'+group+'_'+'median']
        test[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'median'] = test[numeric_var] - test[numeric_var+'_'+group+'_'+'median']
        
        train[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'median'] = np.log(train[numeric_var]+0.00001) / np.log(train[numeric_var+'_'+group+'_'+'median']+0.00001) 
        test[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'median'] = np.log(test[numeric_var]+0.00001) / np.log(test[numeric_var+'_'+group+'_'+'median']+0.00001)
        
        
        a1 = train.groupby([group])[numeric_var].max().to_dict()

        train[numeric_var+'_'+group+'_'+'max'] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+'max'] = test[group].map(a1)

        train[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'max'] = train[numeric_var] - train[numeric_var+'_'+group+'_'+'max']
        test[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'max'] = test[numeric_var] - test[numeric_var+'_'+group+'_'+'max']
        
        train[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'max'] = np.log(train[numeric_var]+0.00001) / np.log(train[numeric_var+'_'+group+'_'+'max']+0.00001)
        test[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'max'] = np.log(test[numeric_var]+0.00001) / np.log(test[numeric_var+'_'+group+'_'+'max']+0.00001)
        
        
        a1 = train.groupby([group])[numeric_var].min().to_dict()

        train[numeric_var+'_'+group+'_'+'min'] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+'min'] = test[group].map(a1)

        train[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'min'] = train[numeric_var] - train[numeric_var+'_'+group+'_'+'min']
        test[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'min'] = test[numeric_var] - test[numeric_var+'_'+group+'_'+'min']
        
        train[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'min'] = np.log(train[numeric_var]+0.00001) / np.log(train[numeric_var+'_'+group+'_'+'min']+0.00001) 
        test[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'min'] = np.log(test[numeric_var]+0.00001) / np.log(test[numeric_var+'_'+group+'_'+'min']+0.00001) 
        
        
        a1 = train.groupby([group])[numeric_var].sum().to_dict()

        train[numeric_var+'_'+group+'_'+'sum'] = train[group].map(a1)
        test[numeric_var+'_'+group+'_'+'sum'] = test[group].map(a1)

        train[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'sum'] = train[numeric_var] - train[numeric_var+'_'+group+'_'+'sum']
        test[numeric_var+'_minus_'+numeric_var+'_'+group+'_'+'sum'] = test[numeric_var] - test[numeric_var+'_'+group+'_'+'sum']
        
        train[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'sum'] = np.log(train[numeric_var]+0.00001) / np.log(train[numeric_var+'_'+group+'_'+'sum']+0.00001)
        test[numeric_var+'_divide_'+numeric_var+'_'+group+'_'+'sum'] = np.log(test[numeric_var]+0.00001) / np.log(test[numeric_var+'_'+group+'_'+'sum']+0.00001)
        

In [13]:
"""
clustetring features
"""

np.random.seed(1)

features = ['ZN','FE','V40','CU','MO']

# sc = MinMaxScaler((0, 1))
# sc_fit = sc.fit(train[features])
# train_scaled = sc_fit.transform(train[features])
# kmeans = KMeans(n_clusters=15,n_init=25, max_iter = 600, random_state=1).fit(train_scaled)
# test_scaled = sc_fit.transform(test[features])
# train["cluster_no"] = kmeans.predict(train_scaled)
# test["cluster_no"] = kmeans.predict(test_scaled)

# 앞에서 min-max 이미 적용 함 
kmeans = KMeans(n_clusters=15,n_init=25, max_iter = 600, random_state=1).fit(train[features])
train["cluster_no"] = kmeans.predict(train[features])
test["cluster_no"] = kmeans.predict(test[features])

# check
# print(train['cluster_no'].value_counts())

for var in ['ANONYMOUS_1','ANONYMOUS_2']+['ANONYMOUS_1_V40', 'ANONYMOUS_1_CR', 'ANONYMOUS_1_ZN','ANONYMOUS_1_FE']:
    
    a1 = train.groupby(['cluster_no'])[var].mean().to_dict()
    train['cluster_no_by'+'_'+var+'_'+'mean'] = train['cluster_no'].map(a1)
    test['cluster_no_by'+'_'+var+'_'+'mean'] = test['cluster_no'].map(a1)
    
    a1 = train.groupby(['cluster_no'])[var].median().to_dict()
    train['cluster_no_by'+'_'+var+'_'+'median'] = train['cluster_no'].map(a1)
    test['cluster_no_by'+'_'+var+'_'+'median'] = test['cluster_no'].map(a1)

    a1 = train.groupby(['cluster_no'])[var].max().to_dict()
    train['cluster_no_by'+'_'+var+'_'+'max'] = train['cluster_no'].map(a1)
    test['cluster_no_by'+'_'+var+'_'+'max'] = test['cluster_no'].map(a1)

    a1 = train.groupby(['cluster_no'])[var].min().to_dict()
    train['cluster_no_by'+'_'+var+'_'+'min'] = train['cluster_no'].map(a1)
    test['cluster_no_by'+'_'+var+'_'+'min'] = test['cluster_no'].map(a1)

In [14]:
# check inf values 
a1 = train.select_dtypes(include=['int','float']).apply(lambda x: x.max(),axis=0).reset_index(name='val')
a1.loc[a1['val']==np.Inf,'index'].tolist()

[]

> 모델학습

In [15]:
def train_model(train,test,params,stratified,num_folds,drop_features,seed_num):
    
    # start log 
    print('-'*50)
    print('>> seed_num:',seed_num)   
    # print('>> drop_features:',len(drop_features))
    
    seed_everything(1)
    
    # Divide in training/validation and test data
    train_df = train.copy()
    test_df = test.copy()

    # label encoding 
    encoder = LabelEncoder()
    categorical_features = [i for i in train_df.select_dtypes(include=['object','category']).columns.tolist() if i not in ['ID']]
    categorical_features = [i for i in categorical_features if i in train_df.columns.tolist()]
    for each in categorical_features:
        train_df[each] = encoder.fit_transform(train_df[each])
        test_df[each] = encoder.fit_transform(test_df[each])

    # set training options
    stratified = stratified
    num_folds = num_folds

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1)

    # Create arrays and dataframes to store results
    oof_preds_lgb = np.zeros(train_df.shape[0])
    sub_preds_lgb = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in ['Y_LABEL','ID','SAMPLE_TRANSFER_DAY']+drop_features]

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['Y_LABEL'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['Y_LABEL'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['Y_LABEL'].iloc[valid_idx]

        """        
        learning_rate=0.02,
        num_leaves=34,
        colsample_bytree=0.2,
        subsample=0.8715623,
        max_depth=4,
        reg_alpha=0.041545473,
        reg_lambda=0.0735294,
        min_split_gain=0.0222415,
        min_child_weight=39.3259775,
        min_child_samples = 10,
        """

        # LightGBM parameters found by Bayesian optimization
        clf = LGBMClassifier(
            
            learning_rate = params['learning_rate'],
            num_leaves = int(round(params['num_leaves'])),
            colsample_bytree = params['colsample_bytree'],
            subsample = params['subsample'],
            max_depth = int(round(params['max_depth'])),
            reg_alpha = params['reg_alpha'],
            reg_lambda = params['reg_lambda'],
            min_split_gain = params['min_split_gain'],
            min_child_weight = params['min_child_weight'],
            min_child_samples = int(round(params['min_child_samples'])),    
            
            n_jobs = -1,
            n_estimators = 10000,            
            random_state = seed_num,
            silent=-1,
            deterministic=True,
            verbose=-1
        )
        
        with warnings.catch_warnings():
            
            warnings.filterwarnings('ignore')

            clf.fit(
                  train_x
                , train_y
                , eval_set=[(train_x, train_y), (valid_x, valid_y)]
                , eval_metric= 'auc'
                , verbose= -1
                , early_stopping_rounds= 500
            )

        oof_preds_lgb[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds_lgb += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        # print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds_lgb[valid_idx])))

    print('Full AUC score %.6f' % roc_auc_score(train_df['Y_LABEL'], oof_preds_lgb))

    # Write submission file and plot feature importance
    test_df['Y_LABEL_lgb'] = sub_preds_lgb

    # vi
    # print('-'*50)
    # display(feature_importance_df.groupby(['feature'])['importance'].sum().sort_values(ascending=False).head(30))
    # print('-'*50)
    # display_importances(feature_importance_df)
    
    # train auc
    oof_auc = roc_auc_score(train_df['Y_LABEL'], oof_preds_lgb)

    
    if oof_auc>=0.701:

        # find the best thred for f1-score
        f1_score_df = pd.DataFrame()
        for thred in [i/10000 for i in range(0,10000,1) if (i/10000>0.1) & (i/10000<0.35)]:

            a1 = pd.DataFrame()
            f1 = f1_score(train_df['Y_LABEL'], np.where(oof_preds_lgb>thred,1,0), average='macro')
            a1['f1'] = [f1]
            a1['thred'] = [thred]
            f1_score_df = pd.concat([f1_score_df, a1], axis=0)

        thred = f1_score_df.loc[f1_score_df['f1']==f1_score_df['f1'].max(),'thred'].tolist()[0]
        print('thred:',thred)
        print('ncol',len(feats))

        # train f1
        print('auc:',oof_auc)
        oof_f1 = f1_score(train_df['Y_LABEL'], np.where(oof_preds_lgb>thred,1,0), average='macro')
        print('f1:',oof_f1)
        a1 = train_df['Y_LABEL'].value_counts()/len(train_df)
        print('Target ratio(real):',(a1[1]))

        # test err
        test_df['TARGET'] = np.where(test_df['Y_LABEL_lgb']>thred,1,0)
        a1 = test_df['TARGET'].value_counts()/len(test_df)
        print('Target ratio(pred):',(a1[1]))
        target_sum = test_df['TARGET'].sum()
        print('Target sum:',target_sum)        
        
        # save 
        train_df['Y_LABEL_lgb'] = oof_preds_lgb
        a1 = train_df[['ID','YEAR','COMPONENT_ARBITRARY','Y_LABEL_lgb','Y_LABEL']].copy()
        a1.to_csv('./data/train_pred_'+str(seed_num)+'_'+str(np.round(oof_f1,10))+'.csv', index= False)    
        a1 = test_df[['ID','YEAR','COMPONENT_ARBITRARY','Y_LABEL_lgb']].copy()
        a1.to_csv('./data/test_pred_'+str(seed_num)+'_'+str(np.round(oof_f1,10))+'.csv', index= False)

        # submit
        
        a1 = test_df[['ID', 'TARGET']].copy()
        a1 = a1.rename(columns={'TARGET':'Y_LABEL'})
        submission_file_name = './data/sample_submission_lgb_'+str(np.round(oof_f1,4))+'.csv'
        a1.to_csv(submission_file_name, index= False)
        
    return test_df,oof_f1

In [16]:
"""
baseline model 
"""

drop_features_vc1 = ['ZN_minus_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_CU', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'FE_divide_FE_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_max', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'FE_minus_FE_YEAR_COMPONENT_median', 'ZN_divide_ZN_YEAR_min', 'PQINDEX_divide_PQINDEX_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_sum', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_2_YEAR_max', 'FE_divide_FE_FE_woe_bin_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'V40_YEAR_median', 'V40_minus_V40_YEAR_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'NI_V40', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_max', 'ZN_minus_ZN_YEAR_min', 'FE_divide_FE_YEAR_COMPONENT_max', 'ZN_divide_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'CU_minus_CU_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_sum', 'ZN_divide_ZN_YEAR_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_max', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'CU_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ZN_YEAR_COMPONENT_sum', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_median', 'CU_divide_CU_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_max', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'FE_divide_FE_FE_woe_bin_max', 'cluster_no_by_ANONYMOUS_1_median', 'V40_divide_V40_FE_woe_bin_sum', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_median', 'V40_divide_V40_YEAR_COMPONENT_max', 'V40_minus_V40_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_median', 'FE_minus_FE_FE_woe_bin_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ZN_minus_ZN_FE_woe_bin_sum', 'CU_minus_CU_FE_woe_bin_sum', 'V40', 'ANONYMOUS_1_ZN_woe_bin', 'FE_divide_FE_COMPONENT_ARBITRARY_min', 'CU_divide_CU_FE_woe_bin_sum', 'cluster_no_by_ANONYMOUS_2_mean', 'CU', 'V40_minus_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'V40_divide_V40_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_sum', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'cluster_no_by_ANONYMOUS_1_V40_mean', 'FE_minus_FE_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_YEAR_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_FE_woe_bin_sum', 'FE_divide_FE_YEAR_sum', 'V40_minus_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_sum', 'ANONYMOUS_1_V40', 'ZN_divide_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_sum', 'ZN_divide_ZN_YEAR_median', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'V40_minus_V40_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CR_FE', 'FE_V40', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'PQINDEX_minus_PQINDEX_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_MO', 'CU_FE', 'ZN_minus_ZN_YEAR_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'V40_divide_V40_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'PQINDEX_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_max', 'CU_ZN', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_minus_V40_FE_woe_bin_min', 'NI_ZN', 'V40_divide_V40_FE_woe_bin_median', 'ZN_minus_ZN_YEAR_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'ANONYMOUS_1_ZN', 'PQINDEX_YEAR_median', 'NI', 'FE_ZN', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'CR_V40', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'MO_V40', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_median', 'FE_minus_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'cluster_no_by_ANONYMOUS_1_V40_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_max', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_median', 'ANONYMOUS_2_MO', 'PQINDEX_log1', 'CU_minus_CU_YEAR_COMPONENT_max', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_max', 'CU_divide_CU_FE_woe_bin_median']
drop_features_vc2 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_CR', 'CU_minus_CU_YEAR_COMPONENT_max', 'CU_minus_CU_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_max', 'FE', 'V40_minus_V40_YEAR_min', 'ZN_divide_ZN_YEAR_COMPONENT_max', 'V40_divide_V40_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_max', 'CU_divide_CU_YEAR_COMPONENT_max', 'NI_ZN', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_sum', 'FE_YEAR_COMPONENT_max', 'FE_divide_FE_YEAR_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_median', 'ZN_divide_ZN_YEAR_COMPONENT_min', 'CU_divide_CU_YEAR_COMPONENT_sum', 'FE_minus_FE_FE_woe_bin_median', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_sum', 'CU_divide_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_median', 'FE_divide_FE_YEAR_COMPONENT_min', 'V40_minus_V40_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_COMPONENT_ARBITRARY_max', 'V40_divide_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_median', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'PQINDEX_minus_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_sum', 'CR_ZN', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_max', 'CR_V40', 'V40_minus_V40_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_sum', 'ZN_divide_ZN_COMPONENT_ARBITRARY_max', 'CU_divide_CU_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_min', 'ZN_minus_ZN_YEAR_COMPONENT_max', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_median', 'FE_minus_FE_YEAR_median', 'ANONYMOUS_1_minus_ANONYMOUS_1_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_max', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_COMPONENT_ARBITRARY_max', 'V40_divide_V40_YEAR_max', 'ANONYMOUS_2_MO', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'V40_divide_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_COMPONENT_ARBITRARY_min', 'PQINDEX_divide_PQINDEX_FE_woe_bin_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_YEAR_COMPONENT_median', 'MO_NI', 'MO_ZN', 'ANONYMOUS_1_minus_ANONYMOUS_1_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_median', 'ZN_minus_ZN_YEAR_COMPONENT_median', 'CR_MO', 'CR_FE', 'ZN_divide_ZN_FE_woe_bin_max', 'V40_minus_V40_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_median', 'FE_divide_FE_YEAR_min', 'PQINDEX_minus_PQINDEX_YEAR_COMPONENT_median', 'V40_woe_bin', 'PQINDEX_divide_PQINDEX_FE_woe_bin_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_median', 'CU_YEAR_COMPONENT_max', 'ANONYMOUS_2_YEAR_COMPONENT_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_min', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_max', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_median', 'ZN_minus_ZN_FE_woe_bin_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_sum', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_min', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_sum', 'FE_NI', 'ANONYMOUS_1_divide_ANONYMOUS_1_FE_woe_bin_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_YEAR_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_sum', 'NI', 'ZN_minus_ZN_YEAR_COMPONENT_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_max', 'ZN_divide_ZN_YEAR_COMPONENT_median', 'V40_YEAR_median', 'PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_min', 'V40_divide_V40_FE_woe_bin_median', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_COMPONENT_max', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_FE_woe_bin_min', 'ANONYMOUS_1_CU', 'CU_divide_CU_FE_woe_bin_max', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_FE_woe_bin_max', 'V40', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_log1', 'ANONYMOUS_2_minus_ANONYMOUS_2_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_YEAR_max', 'V40_divide_V40_FE_woe_bin_min', 'ANONYMOUS_1_FE_divide_ANONYMOUS_1_FE_FE_woe_bin_sum']
drop_features_vc3 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max', 'ANONYMOUS_1_divide_ANONYMOUS_1_COMPONENT_ARBITRARY_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_FE_woe_bin_sum', 'FE_divide_FE_YEAR_COMPONENT_sum', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_COMPONENT_ARBITRARY_median', 'FE_divide_FE_COMPONENT_ARBITRARY_max', 'CU_divide_CU_YEAR_max', 'FE_divide_FE_YEAR_sum', 'V40_divide_V40_YEAR_min', 'FE_divide_FE_FE_woe_bin_max', 'PQINDEX_divide_PQINDEX_YEAR_COMPONENT_median', 'ANONYMOUS_1_V40_minus_ANONYMOUS_1_V40_YEAR_COMPONENT_sum', 'ANONYMOUS_2_divide_ANONYMOUS_2_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_COMPONENT_sum', 'V40_divide_V40_FE_woe_bin_sum', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE_YEAR_median', 'ZN_divide_ZN_YEAR_min', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_min', 'ZN_minus_ZN_FE_woe_bin_sum', 'PQINDEX_divide_PQINDEX_YEAR_min', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_FE_woe_bin_min', 'CU_divide_CU_YEAR_COMPONENT_min', 'ANONYMOUS_2_YEAR_max', 'CU_minus_CU_FE_woe_bin_sum', 'ANONYMOUS_1_FE_minus_ANONYMOUS_1_FE_YEAR_max', 'FE_minus_FE_FE_woe_bin_sum', 'FE_divide_FE_YEAR_max', 'PQINDEX_minus_PQINDEX_FE_woe_bin_max', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_sum', 'FE_minus_FE_YEAR_max', 'ANONYMOUS_1_minus_ANONYMOUS_1_YEAR_COMPONENT_min', 'FE_minus_FE_FE_woe_bin_min', 'cluster_no_by_ANONYMOUS_1_FE_median', 'ANONYMOUS_1_CR_minus_ANONYMOUS_1_CR_YEAR_COMPONENT_median', 'ZN_minus_ZN_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_CR_divide_ANONYMOUS_1_CR_FE_woe_bin_sum', 'ZN_minus_ZN_YEAR_sum', 'NI_V40', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_max', 'CU_minus_CU_YEAR_sum', 'V40_minus_V40_FE_woe_bin_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_median', 'PQINDEX_divide_PQINDEX_FE_woe_bin_max', 'ZN_minus_ZN_FE_woe_bin_max', 'V40_minus_V40_COMPONENT_ARBITRARY_min', 'FE_minus_FE_YEAR_COMPONENT_median', 'ANONYMOUS_1_FE', 'PQINDEX_minus_PQINDEX_FE_woe_bin_median', 'PQINDEX_minus_PQINDEX_YEAR_max', 'FE_YEAR_COMPONENT_median', 'V40_divide_V40_COMPONENT_ARBITRARY_min', 'ANONYMOUS_1_V40_divide_ANONYMOUS_1_V40_FE_woe_bin_sum', 'ZN_minus_ZN_COMPONENT_ARBITRARY_max']
drop_features_vc4 = ['PQINDEX_divide_PQINDEX_COMPONENT_ARBITRARY_median', 'ZN_minus_ZN_YEAR_COMPONENT_min', 'ANONYMOUS_1_ZN_minus_ANONYMOUS_1_ZN_YEAR_sum', 'ANONYMOUS_1_ZN_divide_ANONYMOUS_1_ZN_YEAR_median', 'PQINDEX_divide_PQINDEX_YEAR_median', 'ANONYMOUS_2_minus_ANONYMOUS_2_YEAR_COMPONENT_max', 'FE_divide_FE_FE_woe_bin_min', 'V40_divide_V40_COMPONENT_ARBITRARY_median', 'ANONYMOUS_1_divide_ANONYMOUS_1_YEAR_COMPONENT_max']

params =  {
    'learning_rate': 0.07398,
    'max_depth': 4.309,
    'colsample_bytree': 0.4028,
    'subsample': 0.4278,
    'min_child_samples': 25.65,
    'min_child_weight': 0.6138,
    'min_split_gain': 0.7354,
    'num_leaves': 62.68,
    'reg_alpha': 0.2889,
    'reg_lambda': 7.875
}

# train_model(train,test,params,True,5,drop_features_vc1,seed_num=176)

test_list = []
for i in range(500):
    seed_num = 176+i
    try: 
        a1,oof_f1 = train_model(train,test,params,True,5,drop_features_vc1,seed_num=seed_num)
        a1 = a1[['ID','TARGET']].copy()
        a1 = a1.rename(columns={'TARGET':'Y_LABEL'+str(np.round(oof_f1,4))})
        test_list.append(a1)
    except:
        pass

--------------------------------------------------
>> seed_num: 176
Full AUC score 0.704596
thred: 0.1636
ncol 428
auc: 0.7045962957432151
f1: 0.5922652586883465
Target ratio(real): 0.08534941468605889
Target ratio(pred): 0.08872703194835292
Target sum: 536
--------------------------------------------------
>> seed_num: 177
Full AUC score 0.700338
--------------------------------------------------
>> seed_num: 178
Full AUC score 0.700280
--------------------------------------------------
>> seed_num: 179
Full AUC score 0.701028
thred: 0.1807
ncol 428
auc: 0.7010277079047134
f1: 0.5884145501587312
Target ratio(real): 0.08534941468605889
Target ratio(pred): 0.06753848700546267
Target sum: 408
--------------------------------------------------
>> seed_num: 180
Full AUC score 0.700585
--------------------------------------------------
>> seed_num: 181
Full AUC score 0.701317
thred: 0.168
ncol 428
auc: 0.7013166032586338
f1: 0.5883250877790941
Target ratio(real): 0.08534941468605889
Target 

> seed blending

In [17]:
# read data
path = 'c:/Users/user/Desktop/oil/data'
test_list = []
train_list = []
for file_name in os.listdir(path):
    f = os.path.join(path,file_name)
    if 'test' in f:
        a1 = pd.read_csv(f)
        a1 = a1.rename(columns={'Y_LABEL_lgb':'Y_LABEL_lgb_'+'_'.join(file_name.split('_')[2:4])})
        test_list.append(a1)
    elif 'train' in f:
        a1 = pd.read_csv(f)
        a1 = a1.rename(columns={'Y_LABEL_lgb':'Y_LABEL_lgb_'+'_'.join(file_name.split('_')[2:4])})
        train_list.append(a1)

In [18]:
# merge dataset
train = reduce(lambda x,y:pd.merge(x,y,on=['ID','YEAR','COMPONENT_ARBITRARY','Y_LABEL'],how='left'),train_list)
test = reduce(lambda x,y:pd.merge(x,y,on=['ID','YEAR','COMPONENT_ARBITRARY'],how='left'),test_list)

# drop columns
train = train.drop(columns=['YEAR','COMPONENT_ARBITRARY'])
test = test.drop(columns=['YEAR','COMPONENT_ARBITRARY'])

In [19]:
# 예측값 컬럼 
pred_cols  = sorted(train.columns[train.columns.str.contains('Y_LABEL_')].tolist())

In [20]:
# TARGET=1 blending 값 
a1 = test.copy()
a1[pred_cols] = a1[pred_cols].apply(lambda x: np.where(x>0.2,1,0))
a1['TARGET1'] = a1[pred_cols].apply(lambda x: x.sum(),axis=1)
a1['TARGET1'] = np.where(a1['TARGET1']>0,1,0)
sub_TARGET1 = a1['TARGET1']
print('TARGET1:',a1['TARGET1'].sum())

# TARGET=0 blending 값 
a1 = test.copy()
a1[pred_cols] = a1[pred_cols].apply(lambda x: np.where(x<0.135,1,0))
a1['TARGET0'] = a1[pred_cols].apply(lambda x: x.sum(),axis=1)
a1['TARGET0'] = np.where(a1['TARGET0']>0,1,0)
sub_TARGET0 = a1['TARGET0']
print('TARGET0:',a1['TARGET0'].sum())

TARGET1: 453
TARGET0: 5440


In [21]:
# best모델 호출 
sub = pd.read_csv('c:/Users/user/Desktop/oil/data/sample_submission_lgb_0.5923.csv')
sub['Y_LABEL'].value_counts()

0    5505
1     536
Name: Y_LABEL, dtype: int64

In [22]:
# TARGET=0인 대상 blending값으로 대체 
sub['Y_LABEL'] = np.where(sub_TARGET0>0,0,sub['Y_LABEL'])
sub['Y_LABEL'].value_counts()

0    5523
1     518
Name: Y_LABEL, dtype: int64

In [23]:
# TARGET=1인 대상 blending값으로 대체 
sub['Y_LABEL'] = np.where(sub_TARGET1>0,1,sub['Y_LABEL'])
sub['Y_LABEL'].value_counts()

0    5519
1     522
Name: Y_LABEL, dtype: int64

In [24]:
# 저장
sub.to_csv('c:/Users/user/Desktop/oil/sub_blend_three_model_522.csv',index=False)